# Frame Overlap Tutorial (v0.2.0)

This tutorial demonstrates the frame_overlap package for analyzing neutron Time-of-Flight frame overlap data.

## Overview

The package provides four main classes:

1. **Data**: Load and process ToF data
2. **Reconstruct**: Apply deconvolution filters
3. **Analysis**: Simplified nbragg integration
4. **ParametricScan**: Parameter sensitivity analysis

## CORRECT Workflow Order:

```
Data → Convolute → Poisson → Overlap → Reconstruct → Analysis
        ↓           ↓          ↓          ↓           ↓
    Instrument  Add noise  Frame ops  Recover     Fit with
     response   (+ flux              signal      nbragg
                scaling)
```

**KEY INSIGHT**: Convolute BEFORE Poisson!

Why? The pulse_duration defines the pulsed source duty cycle:
- `effective_flux = flux × pulse_duration × freq`
- Flux is measured for continuous source, but measurement is pulsed
- Convolute stores pulse_duration, Poisson uses it automatically

## New Features in v0.2.0:
- **Automatic flux scaling** by pulse_duration in Poisson
- **tmin/tmax filtering**: Calculate chi² on specific time/wavelength range
- **Vertical indicators**: Show tmin/tmax on plots
- **Two-subplot plotting**: Data + residuals in σ units
- **nbragg integration**: `recon.to_nbragg()` for wavelength conversion
- **Analysis class**: Clean API for nbragg fitting
- **Noise optimization**: `recon.optimize_noise()` with lmfit

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from frame_overlap import Data, Reconstruct, Analysis

%matplotlib inline

## 1. Data Loading

Load signal and openbeam data from CSV files.

In [ ]:
# Load data
data = Data(
    signal_file='iron_powder.csv',
    openbeam_file='openbeam.csv',
    flux=1e6,       # n/cm²/s (continuous source)
    duration=1.0,   # hours
    freq=20,        # Hz
    threshold=0
)

print(data)
print(f"\nLoaded {len(data.table)} data points")
print(f"Time range: {data.table['time'].min():.1f} - {data.table['time'].max():.1f} µs")

In [ ]:
# Visualize raw data
data.plot()
plt.show()

## 2. Instrument Response Convolution (FIRST!)

Apply convolution to simulate the instrument's pulse duration.

**IMPORTANT**: This step stores `pulse_duration`, which will be used by `poisson_sample()` to calculate effective flux.

In [ ]:
# Convolute with 200 µs pulse
data.convolute_response(pulse_duration=200)

print(f"Convolution applied with pulse_duration = {data.pulse_duration} µs")
print(f"Data shape: {data.table.shape}")

In [ ]:
# Visualize convolved data
data.plot(kind="signal",show_stages=True, show_errors=False,ylim=(0,5e5));

## 3. Poisson Sampling (AFTER Convolution!)

Apply Poisson sampling to simulate neutron counting statistics.

**Automatic flux scaling**: Since `pulse_duration` and `freq` are known, the effective flux is:
```
flux_eff = flux × (pulse_duration / 1e6) × freq
         = 1e6 × (200e-6) × 20
         = 4000 n/cm²/s
```

In [ ]:
# Apply Poisson sampling
# flux is automatically scaled by pulse_duration × freq
data.poisson_sample(flux=1e6, measurement_time=240, freq=20)

print("Poisson sampling applied")
print(f"Data shape: {data.table.shape}")

In [ ]:
data.plot(kind="transmission",show_stages=True, show_errors=False,ylim=(0,1));

## 4. Frame Overlap

Create overlapping frames. Example: `[0, 25]` means:
- Frame 1 starts at t = 0 ms
- Frame 2 starts at t = 25 ms

**Optional**: Use `poisson_seed` to add randomness to the overlapped signal with duty_cycle=1.0

In [ ]:
# Create 2-frame overlap
data.overlap(kernel=[0, 25], total_time=50)

print(f"Frame overlap created with kernel: {data.kernel}")
print(f"  Frame 1 starts at: 0 ms")
print(f"  Frame 2 starts at: 25 ms")
print(f"New data length: {len(data.table)}")

In [ ]:
# Plot overlapped data
data.plot(kind="transmission", show_errors=False,ylim=(0,1),xlim=(0,50))
plt.title('Data after Frame Overlap')
plt.show()

In [ ]:
data.plot(kind="signal",show_stages=True, show_errors=False,ylim=(0,5e3));

## 5. Signal Reconstruction

Apply deconvolution to recover the poissoned+convolved signal (before overlap).

**NEW**: Specify `tmin` and `tmax` (in ms) to calculate chi² only on a specific time range!

In [ ]:
# Create Reconstruct object
# Example with tmin/tmax: recon = Reconstruct(data, tmin=10, tmax=40)
recon = Reconstruct(data,tmin=1,tmax=6)

# Apply Wiener filter
recon.filter(kind='wiener', noise_power=0.01)

print("Reconstruction complete!")
print(f"\nWhat we recovered: Poissoned+Convolved signal (before overlap)")
print(f"Reference shape: {recon.reference_data.shape}")
print(f"Reconstructed shape: {recon.reconstructed_data.shape}")

print(f"\nReconstruction Statistics:")
for key, value in recon.get_statistics().items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

### Visualize Reconstruction

Two-subplot layout:
- **Top**: Poissoned+Convolved (target) vs Reconstructed
- **Bottom**: Residuals in σ units

**NEW**: Green/orange vertical lines show tmin/tmax if specified!

In [ ]:
# Plot reconstruction results
recon.plot(kind='transmission', show_errors=False,ylim=(0,1))
plt.show()

In [ ]:
# Plot signal counts
recon.plot(kind='signal', show_errors=False)
plt.show()

### Automatic Noise Optimization

Use `optimize_noise()` to find the optimal noise_power parameter.

In [ ]:
# Optimize noise parameter
try:
    result = recon.optimize_noise(
        kind='wiener',
        noise_min=1e-4,
        noise_max=1.0,
        method='leastsq'
    )
    
    print(f"\nOptimal noise_power: {result.params['noise_power'].value:.4e}")
    print(f"Chi-squared: {result.chisqr:.2f}")
    
    # Plot optimized reconstruction
    recon.plot(kind='transmission', show_errors=True,ylim=(0,1))
    plt.suptitle(f"Optimized (noise={result.params['noise_power'].value:.4e})")
    plt.show()
except ImportError:
    print("lmfit not installed - install with: pip install lmfit")

In [ ]:
result

## 6. nbragg Integration

Convert reconstructed data to nbragg format and fit.

In [ ]:
# Convert to nbragg format
nbragg_data = recon.to_nbragg(L=9.0, tstep=10e-6)
print(f"✓ Converted to nbragg.Data")
print(f"  Type: {type(nbragg_data)}")

In [ ]:
nbragg_data.table.plot(x="wavelength",y="trans",ylim=(0,1))

In [ ]:

analysis = Analysis(xs='iron', vary_background=True,vary_response=True)

print("Analysis object created")
print(f"Access nbragg model: analysis.model.params")

# Fit
result = analysis.fit(recon)

print(f"\nFit complete!")
print(f"Reduced chi-squared: {result.redchi:.3f}")

# Plot
analysis.plot()
plt.show()

In [ ]:
analysis.result

## 7. Method Chaining

Complete workflow in one chain with CORRECT order.

In [ ]:
# Method chaining with CORRECT order
data_chain = (Data(signal_file='iron_powder.csv', 
                   openbeam_file='openbeam.csv',
                   flux=1e6, duration=1.0, freq=20)
              .convolute_response(200)              # 1. Convolute (stores pulse_duration)
              .poisson_sample(flux=1e6,             # 2. Poisson (uses pulse_duration)
                             measurement_time=1.0, 
                             freq=20)
              .overlap([0, 25]))                    # 3. Overlap

print("✓ Method chaining complete (CORRECT ORDER)")
print(data_chain)

# Reconstruct with tmin/tmax filtering
recon_chain = Reconstruct(data_chain, tmin=10, tmax=40).filter(kind='wiener', noise_power=0.01)
chi2_formatted = recon_chain._format_chi2(recon_chain.statistics.get('chi2_per_dof'))
print(f"\nχ²/dof (10-40 ms range): {chi2_formatted}")

## Summary

**CORRECT Processing Order**: Data → **Convolute** → **Poisson** → Overlap → Reconstruct → Analysis

**Key Points**:
1. **Convolute FIRST** to define pulse_duration
2. **Poisson SECOND** uses pulse_duration for flux scaling:
   - `flux_eff = flux × (pulse_duration / 1e6) × freq`
3. **Overlap THIRD** creates overlapped frames
4. **Reconstruction** recovers poissoned+convolved signal (before overlap)
5. **Optional 2nd Poisson**: Use `overlap(poisson_seed=42)` for randomness

**NEW v0.2.0 Features**:
- Automatic pulse_duration scaling in Poisson
- tmin/tmax for chi² range filtering
- Vertical indicators on plots
- `recon.to_nbragg()` for wavelength conversion
- `recon.optimize_noise()` with lmfit
- Analysis class: `analysis.model.params`

**Why This Order?**
Flux is measured for continuous source, but neutron measurements use pulsed sources. The pulse_duration (from convolution) defines the pulsing duty cycle, which must be applied to the flux before Poisson sampling.